In [2]:
import csv
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import load_model

Using TensorFlow backend.


In [0]:
START = "^"
END = "$"
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
num_samples = 10000  # Number of samples to train on.
latent_dim = 256
DATA_PATH = './drive/My Drive/Colab Notebooks/12-data'

def get_file_data(path):
    input_texts, target_texts = [], []
    input_characters, target_characters = set(), set()
    with open(path) as f:
        reader = csv.reader(f, dialect='excel-tab')
        for i, row in enumerate(reader):
            input_text = START + row[1] + END
            target_text = START + row[2] + END
            if i < num_samples:
                input_texts.append(input_text)
                input_texts.append(target_text)
                target_texts.append(target_text)
                target_texts.append(input_text)
                for char in input_text:
                  if char not in input_characters:
                      input_characters.add(char)
                  if char not in target_characters:
                      target_characters.add(char)
                for char in target_text:
                  if char not in target_characters:
                      target_characters.add(char)
                  if char not in input_characters:
                      input_characters.add(char)
        input_characters = sorted(list(input_characters))
        target_characters = sorted(list(target_characters))
    return input_texts, input_characters, target_texts, target_characters

In [0]:
input_texts, input_characters, target_texts, target_characters = get_file_data(f'{DATA_PATH}/en-train-100K.txt')
# dev_src, dev_target = get_file_data(f'{DATA_PATH}/en-dev.txt')
# test_src, test_target = get_file_data(f'{DATA_PATH}/en-test.txt')

In [27]:
print(len(input_texts))
print(input_texts[10:13])
print(target_texts[10:13])

20000
['^Hyah ! Hmm .$', '^Wiggle your big toe .$', '^He believes in you .$']
['^Wiggle your big toe .$', '^Hyah ! Hmm .$', '^He has faith in you .$']


In [28]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 111
Number of unique output tokens: 111
Max sequence length for inputs: 126
Max sequence length for outputs: 126


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [31]:
print('Encoder input shape:', encoder_input_data.shape)
print('Decoder input shape:', decoder_input_data.shape)
print('Decoder taget shape:', decoder_target_data.shape)

Encoder input shape: (20000, 126, 111)
Decoder input shape: (20000, 126, 111)
Decoder taget shape: (20000, 126, 111)


In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [37]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save(f'{DATA_PATH}/s2s.h5')

Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 252s 16ms/step - loss: 0.3027 - accuracy: 0.9134 - val_loss: 0.2742 - val_accuracy: 0.9213
Epoch 2/50
16000/16000 [==============================] - 251s 16ms/step - loss: 0.2704 - accuracy: 0.9229 - val_loss: 0.2549 - val_accuracy: 0.9267
Epoch 3/50
16000/16000 [==============================] - 250s 16ms/step - loss: 0.2493 - accuracy: 0.9282 - val_loss: 0.2391 - val_accuracy: 0.9302
Epoch 4/50
16000/16000 [==============================] - 250s 16ms/step - loss: 0.2353 - accuracy: 0.9327 - val_loss: 0.2269 - val_accuracy: 0.9338
Epoch 5/50
16000/16000 [==============================] - 248s 16ms/step - loss: 0.2215 - accuracy: 0.9360 - val_loss: 0.2184 - val_accuracy: 0.9364
Epoch 6/50
16000/16000 [==============================] - 248s 15ms/step - loss: 0.2104 - accuracy: 0.9390 - val_loss: 0.2107 - val_accuracy: 0.9387
Epoch 7/50
16000/16000 [==============================] -

In [77]:
print(model.outputs)
print(decoder_outputs)

[<tf.Tensor 'dense_2/truediv:0' shape=(None, None, 111) dtype=float32>]
Tensor("dense_2_1/truediv:0", shape=(None, None, 111), dtype=float32)


In [0]:
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[START]] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == END or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [42]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ^Jumby now wants to be born .$
Decoded sentence: And that 's all I know .$
-
Input sentence: ^Jumby want birth .$
Decoded sentence: Take off your jacket .$
-
Input sentence: ^It was a difficult and long delivery .$
Decoded sentence: It 's not gonna be easy .$
-
Input sentence: ^The delivery was difficult and long .$
Decoded sentence: The situation you .$
-
Input sentence: ^I like to be beautiful everyday .$
Decoded sentence: I was thinking about you .$
-
Input sentence: ^I like to be pretty everyday .$
Decoded sentence: I was thinking about you .$
-
Input sentence: ^Bernadette wants a prenup .$
Decoded sentence: Good morning , darling .$
-
Input sentence: ^Bernadette wants to get a prenup .$
Decoded sentence: Good morning , darling .$
-
Input sentence: ^Don 't say you don 't remember me .$
Decoded sentence: Don 't forget that .$
-
Input sentence: ^Don 't tell me you don 't remember me .$
Decoded sentence: Don 't forget that .$


In [0]:
### eval

In [0]:
model = load_model(f'{DATA_PATH}/s2s.h5')

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 82)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 101)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 347136      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  366592      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [84]:
!pip install py-rouge
import rouge

In [85]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
def prepare_results(metric, p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


def evaluate(hypothesis, references):
  for aggregator in ['Avg', 'Best', 'Individual']:
      print('Evaluation with {}'.format(aggregator))
      apply_avg = aggregator == 'Avg'
      apply_best = aggregator == 'Best'

      evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                            max_n=4,
                            limit_length=True,
                            length_limit=100,
                            length_limit_type='words',
                            apply_avg=apply_avg,
                            apply_best=apply_best,
                            alpha=0.5, # Default F1_score
                            weight_factor=1.2,
                            stemming=True)

      scores = evaluator.get_scores(hypothesis, references)

      for metric, results in sorted(scores.items(), key=lambda x: x[0]):
          if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
              for hypothesis_id, results_per_ref in enumerate(results):
                  nb_references = len(results_per_ref['p'])
                  for reference_id in range(nb_references):
                      print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                      print('\t' + prepare_results(metric, results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
              print()
          else:
              print(prepare_results(metric, results['p'], results['r'], results['f']))
      print()

In [0]:
def get_evaluation_data(input_texts):
  hypothesis, references = [], []
  for seq_index in range(100):
      # Take one sequence (part of the training set)
      # for trying out decoding.
      input_seq = encoder_input_data[seq_index: seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      hypothesis.append(input_texts[seq_index])
      references.append(decoded_sentence)

  return hypothesis, references

In [0]:
hypothesis, references = get_evaluation_data(input_texts)

In [98]:
evaluate(hypothesis, references)

Evaluation with Avg
	rouge-1:	P: 29.15	R: 22.47	F1: 24.52
	rouge-2:	P: 13.33	R:  9.18	F1: 10.30
	rouge-3:	P:  5.08	R:  3.18	F1:  3.68
	rouge-4:	P:  0.67	R:  0.83	F1:  0.73
	rouge-l:	P: 33.30	R: 26.89	F1: 29.03
	rouge-w:	P: 28.35	R: 15.42	F1: 19.18

Evaluation with Best
	rouge-1:	P: 29.15	R: 22.47	F1: 24.52
	rouge-2:	P: 13.33	R:  9.18	F1: 10.30
	rouge-3:	P:  5.08	R:  3.18	F1:  3.68
	rouge-4:	P:  0.67	R:  0.83	F1:  0.73
	rouge-l:	P: 33.30	R: 26.89	F1: 29.03
	rouge-w:	P: 28.35	R: 15.42	F1: 19.18

Evaluation with Individual
	Hypothesis #0 & Reference #0: 
		rouge-1:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #1 & Reference #0: 
		rouge-1:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #2 & Reference #0: 
		rouge-1:	P: 14.29	R: 14.29	F1: 14.29
	Hypothesis #3 & Reference #0: 
		rouge-1:	P: 16.67	R: 33.33	F1: 22.22
	Hypothesis #4 & Reference #0: 
		rouge-1:	P: 16.67	R: 20.00	F1: 18.18
	Hypothesis #5 & Reference #0: 
		rouge-1:	P: 16.67	R: 20.00	F1: 18.18
	Hypothesis #6 & Reference #0: 
		rouge-1:	P:  0